In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import seaborn as sns


In [2]:
# Load the dataset
df = pd.read_csv("stackoverflow_full.csv")
df=df.dropna() #drop null values
df.drop(["Unnamed: 0"], inplace = True, axis =1 )
df.drop(columns = 'HaveWorkedWith', inplace=True) #mostly redundant with computerskills, it would be useful if I was breaking things down by individual programming languages

df.isnull().sum()



Age               0
Accessibility     0
EdLevel           0
Employment        0
Gender            0
MentalHealth      0
MainBranch        0
YearsCode         0
YearsCodePro      0
Country           0
PreviousSalary    0
ComputerSkills    0
Employed          0
dtype: int64

In [3]:
# Encode categorical variables
label_encoders = {}
for col in ['Age', 'Accessibility', 'MentalHealth', 'EdLevel', 'Gender', 'MainBranch', 'Country']:  
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store encoder for future transformations


df.head()

,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,ComputerSkills,Employed
0,0,0,0,1,0,0,0,7,4,146,51552.0,4,0
1,0,0,4,1,0,0,0,12,5,141,46482.0,12,1
2,0,0,0,1,0,0,0,15,6,54,77290.0,7,0
3,0,0,4,1,0,0,0,9,6,27,46135.0,13,0
4,1,0,3,0,0,0,1,40,30,135,160932.0,2,0


In [4]:
# Define new predictor variables (excluding non-significant ones)
significant_predictors = ['EdLevel', 'Gender', 'YearsCodePro', 'PreviousSalary']

# Define new independent variable set
X = df[significant_predictors]
y = df['Employed']
